In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [2]:
mx.context.num_gpus()

0

In [3]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/Merged-update_hourly.csv')
df.fillna(0, inplace=True)

In [4]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,2.90,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0


In [5]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'Time'], infer_datetime_format=True)

In [6]:
df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
0,1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,...,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0,2010-01-01 00:00:00
1,1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,...,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0,2010-01-01 01:00:00
2,1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,...,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0,2010-01-01 02:00:00
3,1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,...,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0,2010-01-01 03:00:00
4,1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,...,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0,2010-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.180,0.0,0.0,2020-12-31 22:00:00


In [7]:
df.columns

Index(['Time', 'WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
       'TWS_S25A', 'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
       'TWS_S25B', 'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'TWS_S26', 'PUMP_S26', 'MEAN_RAIN', 'timestamp'],
      dtype='object')

In [8]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [9]:
train_df.shape

(77145, 21)

In [10]:
test_df

,Time,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,...,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN,timestamp
77145,10/20/18 09:00,1.06,1.01,0.0,0.0,2.41,1.07,0.0,0.0,0.0,...,1.15,0.0,469.19,0.0,2.61,2.35,1.11,0.0,0.0,2018-10-20 09:00:00
77146,10/20/18 10:00,0.68,0.63,0.0,0.0,2.41,0.71,0.0,0.0,0.0,...,0.82,0.0,0.00,0.0,0.00,2.62,0.72,0.0,0.0,2018-10-20 10:00:00
77147,10/20/18 11:00,0.31,0.32,0.0,0.0,2.41,0.33,0.0,0.0,0.0,...,0.42,0.0,0.00,0.0,0.00,2.62,0.37,0.0,0.0,2018-10-20 11:00:00
77148,10/20/18 12:00,0.20,0.23,0.0,0.0,2.40,0.24,0.0,0.0,0.0,...,0.35,0.0,0.00,0.0,0.00,2.67,0.28,0.0,0.0,2018-10-20 12:00:00
77149,10/20/18 13:00,0.41,0.42,0.0,0.0,2.39,0.42,0.0,0.0,0.0,...,0.48,0.0,0.00,0.0,0.00,2.72,0.45,0.0,0.0,2018-10-20 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96427,12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,...,1.31,0.0,456.34,0.0,2.61,2.49,1.29,0.0,0.0,2020-12-31 19:00:00
96428,12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,...,1.68,0.0,380.19,0.0,2.62,2.48,1.66,0.0,0.0,2020-12-31 20:00:00
96429,12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,...,2.04,0.0,287.12,0.0,2.62,2.54,2.04,0.0,0.0,2020-12-31 21:00:00
96430,12/31/20 22:00,2.13,2.03,0.0,0.0,1.94,2.13,0.0,0.0,0.0,...,2.21,0.0,253.27,0.0,2.62,2.58,2.18,0.0,0.0,2020-12-31 22:00:00


In [18]:
train_ds = PandasDataset(
    train_df,
    target = 'TWS_S25A',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
        'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
        'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN',],
    freq = 'H'
)

In [19]:
test_ds = PandasDataset(
    test_df,
    target = 'TWS_S25A',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
        'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
        'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
    freq = 'H'
)

#### training

In [22]:
K = 24
n_hours = 24*3

In [23]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=150,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-08)

estimator = DeepAREstimator(
    prediction_length=K,
    freq="H",
    distr_output = NegativeBinomialOutput(),
    use_feat_dynamic_real=True,
    batch_size=32,
    trainer=trainer,
    dropout_rate=0.4
)

model = estimator.train(train_ds)

  0%|                                                                                               | 0/100 [00:00<?, ?it/s]WARNING:gluonts.trainer:Batch [2] of Epoch[16] gave NaN loss and it will be ignored


 44%|██████████████████████                            | 44/100 [01:57<02:32,  2.73s/it, epoch=49/150, avg_epoch_loss=0.448]WARNING:gluonts.trainer:Batch [46] of Epoch[48] gave NaN loss and it will be ignored


 93%|██████████████████████████████████████████████▌   | 93/100 [04:45<00:19,  2.83s/it, epoch=84/150, avg_epoch_loss=0.448]WARNING:gluonts.trainer:Batch [95] of Epoch[83] gave NaN loss and it will be ignored


 81%|███████████████████████████████████████▋         | 81/100 [04:02<00:59,  3.11s/it, epoch=114/150, avg_epoch_loss=0.449]WARNING:gluonts.trainer:Batch [83] of Epoch[113] gave NaN loss and it will be ignored


  3%|█▌                                                | 3/100 [00:10<05:36,  3.47s/it, epoch=116/150, avg_epoch_loss=0.412]
/lclhome/jshi008/miniconda3/envs/tf3/lib/python3.8/site-packages/gluonts/mx/trainer/_base.py:474: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(


#### predicted y

In [24]:
# forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=500)
# forecasts = list(forecast_it)
# tss = list(ts_it)
# evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
# agg_metrics, item_metrics = evaluator(tss, forecasts)

In [25]:
quantiles = ['0.025', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.975']
q50 = 5

In [26]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'TWS_S25A',
    timestamp = 'timestamp',
    feat_dynamic_real = ['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A',
        'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B',
        'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2',
       'HWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].samples
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[2.3499999  2.06500006 1.57000005 0.95499998 0.53500003 0.37
 0.55000001 1.00999999 1.48000002 2.02999997 2.45000005 2.80500007
 2.51999998 2.19499993 1.58500004 1.         0.55500001 0.23999999
 0.33500001 0.79000002 1.27999997 1.96000004 2.125      2.63000011]
(24,)
------
1
[2.3499999  2.06500006 1.57000005 0.95499998 0.53500003 0.37
 0.55000001 1.00999999 1.48000002 2.02999997 2.45000005 2.80500007
 2.51999998 2.19499993 1.58500004 1.         0.55500001 0.23999999
 0.33500001 0.79000002 1.27999997 1.96000004 2.125      2.63000011
 2.29999995 1.58500004 0.97500002 0.66000003 0.44999999 0.55000001
 0.94999999 1.47500002 2.0250001  2.375      2.66000009 2.73000002
 2.25999999 1.41999996 0.98500001 0.61000001 0.28999999 0.41
 0.815      1.375      1.73500001 2.2750001  2.96499991 2.71499991]
(48,)
------
2
[2.3499999  2.06500006 1.57000005 0.95499998 0.53500003 0.37
 0.55000001 1.00999999 1.48000002 2.02999997 2.45000005 2.80500007
 2.51999998 2.19499993 1.58500004 1.         0.55500

[2.3499999  2.06500006 1.57000005 0.95499998 0.53500003 0.37
 0.55000001 1.00999999 1.48000002 2.02999997 2.45000005 2.80500007
 2.51999998 2.19499993 1.58500004 1.         0.55500001 0.23999999
 0.33500001 0.79000002 1.27999997 1.96000004 2.125      2.63000011
 2.29999995 1.58500004 0.97500002 0.66000003 0.44999999 0.55000001
 0.94999999 1.47500002 2.0250001  2.375      2.66000009 2.73000002
 2.25999999 1.41999996 0.98500001 0.61000001 0.28999999 0.41
 0.815      1.375      1.73500001 2.2750001  2.96499991 2.71499991
 1.67999995 1.02999997 0.51499999 0.41       0.58999997 1.07500005
 1.49000001 2.13499999 2.38499999 2.74000001 2.53999996 2.03500009
 1.625      1.02999997 0.68000001 0.285      0.36000001 0.76499999
 1.26999998 1.995      2.14499998 2.6400001  2.81999993 2.19499993
 1.06500006 0.59500003 0.40000001 0.51999998 0.93000001 1.45000005
 2.06999993 2.39499998 2.58500004 2.58500004 2.25999999 1.46500003
 1.05999994 0.57499999 0.23999999 0.285      0.66500002 1.35000002
 1.8099

[2.3499999  2.06500006 1.57000005 0.95499998 0.53500003 0.37
 0.55000001 1.00999999 1.48000002 2.02999997 2.45000005 2.80500007
 2.51999998 2.19499993 1.58500004 1.         0.55500001 0.23999999
 0.33500001 0.79000002 1.27999997 1.96000004 2.125      2.63000011
 2.29999995 1.58500004 0.97500002 0.66000003 0.44999999 0.55000001
 0.94999999 1.47500002 2.0250001  2.375      2.66000009 2.73000002
 2.25999999 1.41999996 0.98500001 0.61000001 0.28999999 0.41
 0.815      1.375      1.73500001 2.2750001  2.96499991 2.71499991
 1.67999995 1.02999997 0.51499999 0.41       0.58999997 1.07500005
 1.49000001 2.13499999 2.38499999 2.74000001 2.53999996 2.03500009
 1.625      1.02999997 0.68000001 0.285      0.36000001 0.76499999
 1.26999998 1.995      2.14499998 2.6400001  2.81999993 2.19499993
 1.06500006 0.59500003 0.40000001 0.51999998 0.93000001 1.45000005
 2.06999993 2.39499998 2.58500004 2.58500004 2.25999999 1.46500003
 1.05999994 0.57499999 0.23999999 0.285      0.66500002 1.35000002
 1.8099

[2.3499999  2.06500006 1.57000005 0.95499998 0.53500003 0.37
 0.55000001 1.00999999 1.48000002 2.02999997 2.45000005 2.80500007
 2.51999998 2.19499993 1.58500004 1.         0.55500001 0.23999999
 0.33500001 0.79000002 1.27999997 1.96000004 2.125      2.63000011
 2.29999995 1.58500004 0.97500002 0.66000003 0.44999999 0.55000001
 0.94999999 1.47500002 2.0250001  2.375      2.66000009 2.73000002
 2.25999999 1.41999996 0.98500001 0.61000001 0.28999999 0.41
 0.815      1.375      1.73500001 2.2750001  2.96499991 2.71499991
 1.67999995 1.02999997 0.51499999 0.41       0.58999997 1.07500005
 1.49000001 2.13499999 2.38499999 2.74000001 2.53999996 2.03500009
 1.625      1.02999997 0.68000001 0.285      0.36000001 0.76499999
 1.26999998 1.995      2.14499998 2.6400001  2.81999993 2.19499993
 1.06500006 0.59500003 0.40000001 0.51999998 0.93000001 1.45000005
 2.06999993 2.39499998 2.58500004 2.58500004 2.25999999 1.46500003
 1.05999994 0.57499999 0.23999999 0.285      0.66500002 1.35000002
 1.8099

[2.3499999  2.06500006 1.57000005 0.95499998 0.53500003 0.37
 0.55000001 1.00999999 1.48000002 2.02999997 2.45000005 2.80500007
 2.51999998 2.19499993 1.58500004 1.         0.55500001 0.23999999
 0.33500001 0.79000002 1.27999997 1.96000004 2.125      2.63000011
 2.29999995 1.58500004 0.97500002 0.66000003 0.44999999 0.55000001
 0.94999999 1.47500002 2.0250001  2.375      2.66000009 2.73000002
 2.25999999 1.41999996 0.98500001 0.61000001 0.28999999 0.41
 0.815      1.375      1.73500001 2.2750001  2.96499991 2.71499991
 1.67999995 1.02999997 0.51499999 0.41       0.58999997 1.07500005
 1.49000001 2.13499999 2.38499999 2.74000001 2.53999996 2.03500009
 1.625      1.02999997 0.68000001 0.285      0.36000001 0.76499999
 1.26999998 1.995      2.14499998 2.6400001  2.81999993 2.19499993
 1.06500006 0.59500003 0.40000001 0.51999998 0.93000001 1.45000005
 2.06999993 2.39499998 2.58500004 2.58500004 2.25999999 1.46500003
 1.05999994 0.57499999 0.23999999 0.285      0.66500002 1.35000002
 1.8099

...

[2.3499999  2.06500006 1.57000005 ... 0.995      1.66999996 2.06500006]
(450744,)
------
18781
[2.3499999  2.06500006 1.57000005 ... 1.55999994 2.17499995 2.3900001 ]
(450768,)
------
18782
[2.3499999  2.06500006 1.57000005 ... 2.17499995 2.13499999 2.19000006]
(450792,)
------
18783
[2.3499999  2.06500006 1.57000005 ... 2.38000011 2.21499991 2.00999999]
(450816,)
------
18784
[2.3499999  2.06500006 1.57000005 ... 2.00999999 1.66499996 1.10000002]
(450840,)
------
18785
[2.3499999  2.06500006 1.57000005 ... 1.87       1.30999994 0.58499998]
(450864,)
------
18786
[2.3499999  2.06500006 1.57000005 ... 1.41999996 0.67500001 0.06      ]
(450888,)
------
18787
[2.3499999  2.06500006 1.57000005 ... 0.57499999 0.08       0.01      ]
(450912,)
------
18788
[2.3499999  2.06500006 1.57000005 ... 0.09       0.         0.        ]
(450936,)
------
18789
[2.3499999  2.06500006 1.57000005 ... 0.         0.         0.14      ]
(450960,)
------
18790
[2.3499999  2.06500006 1.57000005 ... 0.005      0

[2.3499999  2.06500006 1.57000005 ... 1.04499996 1.45000005 1.995     ]
(452832,)
------
18868
[2.3499999  2.06500006 1.57000005 ... 1.63499999 2.00500011 2.29500008]
(452856,)
------
18869
[2.3499999  2.06500006 1.57000005 ... 1.95000005 2.26999998 2.21000004]
(452880,)
------
18870
[2.3499999  2.06500006 1.57000005 ... 2.10500002 2.06999993 1.78999996]
(452904,)
------
18871
[2.3499999  2.06500006 1.57000005 ... 2.25       1.85000002 1.30999994]
(452928,)
------
18872
[2.3499999  2.06500006 1.57000005 ... 1.92499995 1.52999997 0.755     ]
(452952,)
------
18873
[2.3499999  2.06500006 1.57000005 ... 1.35500002 0.94       0.47499999]
(452976,)
------
18874
[2.3499999  2.06500006 1.57000005 ... 0.94       0.39500001 0.155     ]
(453000,)
------
18875
[2.3499999  2.06500006 1.57000005 ... 0.42500001 0.205      0.31999999]
(453024,)
------
18876
[2.3499999  2.06500006 1.57000005 ... 0.23999999 0.30000001 0.54500002]
(453048,)
------
18877
[2.3499999  2.06500006 1.57000005 ... 0.28       0

[2.3499999  2.06500006 1.57000005 ... 1.255      1.65999997 1.995     ]
(454920,)
------
18955
[2.3499999  2.06500006 1.57000005 ... 1.67999995 1.95000005 2.39499998]
(454944,)
------
18956
[2.3499999  2.06500006 1.57000005 ... 1.90999997 2.17499995 1.98000002]
(454968,)
------
18957
[2.3499999  2.06500006 1.57000005 ... 2.00999999 2.23000002 1.84000003]
(454992,)
------
18958
[2.3499999  2.06500006 1.57000005 ... 1.98500001 1.63499999 1.38      ]
(455016,)
------
18959
[2.3499999  2.06500006 1.57000005 ... 1.70500004 1.33500004 0.86500001]
(455040,)
------
18960
[2.3499999  2.06500006 1.57000005 ... 1.47500002 0.995      0.73000002]
(455064,)
------
18961
[2.3499999  2.06500006 1.57000005 ... 1.10500002 0.815      0.44999999]
(455088,)
------
18962
[2.3499999  2.06500006 1.57000005 ... 0.67500001 0.58499998 0.55000001]
(455112,)
------
18963
[2.3499999  2.06500006 1.57000005 ... 0.63999999 0.64499998 0.77999997]
(455136,)
------
18964
[2.3499999  2.06500006 1.57000005 ... 0.51999998 0

[2.3499999  2.06500006 1.57000005 ... 1.17499995 1.51999998 1.88999999]
(457008,)
------
19042
[2.3499999  2.06500006 1.57000005 ... 1.46500003 1.75       1.88499999]
(457032,)
------
19043
[2.3499999  2.06500006 1.57000005 ... 1.75999999 1.83500004 1.68499994]
(457056,)
------
19044
[2.3499999  2.06500006 1.57000005 ... 1.90999997 1.71500003 1.33500004]
(457080,)
------
19045
[2.3499999  2.06500006 1.57000005 ... 1.65499997 1.58000004 1.17999995]
(457104,)
------
19046
[2.3499999  2.06500006 1.57000005 ... 1.48000002 1.17499995 0.70499998]
(457128,)
------
19047
[2.3499999  2.06500006 1.57000005 ... 1.16999996 0.70499998 0.38999999]
(457152,)
------
19048
[2.3499999  2.06500006 1.57000005 ... 0.71499997 0.39500001 0.23999999]
(457176,)
------
19049
[2.3499999  2.06500006 1.57000005 ... 0.45500001 0.25999999 0.29499999]
(457200,)
------
19050
[2.3499999  2.06500006 1.57000005 ... 0.175      0.34       0.58999997]
(457224,)
------
19051
[2.3499999  2.06500006 1.57000005 ... 0.38       0

[2.3499999  2.06500006 1.57000005 ... 1.20500004 1.61000001 1.84500003]
(459096,)
------
19129
[2.3499999  2.06500006 1.57000005 ... 1.44500005 1.79499996 1.97500002]
(459120,)
------
19130
[2.3499999  2.06500006 1.57000005 ... 1.76999998 1.86500001 1.91499996]
(459144,)
------
19131
[2.3499999  2.06500006 1.57000005 ... 1.90499997 1.90499997 1.48500001]
(459168,)
------
19132
[2.3499999  2.06500006 1.57000005 ... 1.95000005 1.48500001 0.97500002]
(459192,)
------
19133
[2.3499999  2.06500006 1.57000005 ... 1.49000001 1.14999998 0.61000001]
(459216,)
------
19134
[2.3499999  2.06500006 1.57000005 ... 1.04499996 0.55000001 0.345     ]
(459240,)
------
19135
[2.3499999  2.06500006 1.57000005 ... 0.63999999 0.26499999 0.25999999]
(459264,)
------
19136
[2.3499999  2.06500006 1.57000005 ... 0.38999999 0.27500001 0.38499999]
(459288,)
------
19137
[2.3499999  2.06500006 1.57000005 ... 0.22499999 0.435      0.73000002]
(459312,)
------
19138
[2.3499999  2.06500006 1.57000005 ... 0.36000001 0

In [27]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(19192, 24)

In [28]:
test_ds_predict_q50_supervised[0:5]

array([[2.3499999 , 2.06500006, 1.57000005, 0.95499998, 0.53500003,
        0.37      , 0.55000001, 1.00999999, 1.48000002, 2.02999997,
        2.45000005, 2.80500007, 2.51999998, 2.19499993, 1.58500004,
        1.        , 0.55500001, 0.23999999, 0.33500001, 0.79000002,
        1.27999997, 1.96000004, 2.125     , 2.63000011],
       [2.29999995, 1.58500004, 0.97500002, 0.66000003, 0.44999999,
        0.55000001, 0.94999999, 1.47500002, 2.0250001 , 2.375     ,
        2.66000009, 2.73000002, 2.25999999, 1.41999996, 0.98500001,
        0.61000001, 0.28999999, 0.41      , 0.815     , 1.375     ,
        1.73500001, 2.2750001 , 2.96499991, 2.71499991],
       [1.67999995, 1.02999997, 0.51499999, 0.41      , 0.58999997,
        1.07500005, 1.49000001, 2.13499999, 2.38499999, 2.74000001,
        2.53999996, 2.03500009, 1.625     , 1.02999997, 0.68000001,
        0.285     , 0.36000001, 0.76499999, 1.26999998, 1.995     ,
        2.14499998, 2.6400001 , 2.81999993, 2.19499993],
       [1.065

#### actual y

In [36]:
from helper import series_to_supervised, stage_series_to_supervised

In [37]:
test_df_supervised = series_to_supervised(test_df[['TWS_S25A']], n_hours, K)

In [38]:
test_df_supervised.iloc[:, -K:].shape

(19192, 24)

In [39]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [40]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),...,var1(t+14),var1(t+15),var1(t+16),var1(t+17),var1(t+18),var1(t+19),var1(t+20),var1(t+21),var1(t+22),var1(t+23)
77217,2.56,1.94,1.48,1.03,0.54,0.23,0.51,0.98,1.61,2.14,...,1.49,0.95,0.46,0.10,0.23,0.68,1.34,1.92,2.41,2.75
77218,1.94,1.48,1.03,0.54,0.23,0.51,0.98,1.61,2.14,2.54,...,0.95,0.46,0.10,0.23,0.68,1.34,1.92,2.41,2.75,2.78
77219,1.48,1.03,0.54,0.23,0.51,0.98,1.61,2.14,2.54,2.76,...,0.46,0.10,0.23,0.68,1.34,1.92,2.41,2.75,2.78,2.34
77220,1.03,0.54,0.23,0.51,0.98,1.61,2.14,2.54,2.76,2.62,...,0.10,0.23,0.68,1.34,1.92,2.41,2.75,2.78,2.34,1.79
77221,0.54,0.23,0.51,0.98,1.61,2.14,2.54,2.76,2.62,2.08,...,0.23,0.68,1.34,1.92,2.41,2.75,2.78,2.34,1.79,1.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96404,1.95,2.21,2.12,1.69,1.24,0.73,0.15,-0.15,-0.18,0.11,...,2.35,2.21,1.73,1.19,0.63,0.13,0.04,0.16,0.74,1.25
96405,2.21,2.12,1.69,1.24,0.73,0.15,-0.15,-0.18,0.11,0.78,...,2.21,1.73,1.19,0.63,0.13,0.04,0.16,0.74,1.25,1.64
96406,2.12,1.69,1.24,0.73,0.15,-0.15,-0.18,0.11,0.78,1.34,...,1.73,1.19,0.63,0.13,0.04,0.16,0.74,1.25,1.64,2.00
96407,1.69,1.24,0.73,0.15,-0.15,-0.18,0.11,0.78,1.34,1.80,...,1.19,0.63,0.13,0.04,0.16,0.74,1.25,1.64,2.00,2.13


#### performance on test set

In [41]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 0.191
RMSE = 0.227
